In [1]:
from espnet2.tasks.asr_TruCLeS import ASRTask
from pathlib import Path
import yaml
import argparse
import torch

model_tool = ASRTask

In [2]:
import torchaudio

test_dataset = torchaudio.datasets.LIBRISPEECH("./", url="test-clean", download=True)

In [3]:
path_str = '/home/royshivam/anaconda3/envs/espnet/lib/python3.8/site-packages/espnet_model_zoo/models--pyf98--librispeech_100_e_branchformer/snapshots/e331ba7b520392b7e290b9c73bf1453967a440d2/exp/asr_train_asr_e_branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0_raw_en_bpe5000_sp/config.yaml'
model_path = '/home/royshivam/anaconda3/envs/espnet/lib/python3.8/site-packages/espnet_model_zoo/models--pyf98--librispeech_100_e_branchformer/snapshots/e331ba7b520392b7e290b9c73bf1453967a440d2/exp/asr_train_asr_e_branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0_raw_en_bpe5000_sp/valid.acc.ave_10best.pth'

config_file = Path(path_str)
with config_file.open("r", encoding="utf-8") as f:
    args = yaml.safe_load(f)

args = argparse.Namespace(**args)
model = model_tool.build_model(args)

state_dict = torch.load(model_path, map_location='cpu')


model.to('cpu')
use_lora = getattr(args, "use_lora", False)
model.load_state_dict(state_dict, strict=not use_lora)
if any(["frontend.upstream.model" in k for k in state_dict.keys()]):
    if any(
        [
            "frontend.upstream.upstream.model" in k
            for k in dict(model.named_parameters())
                        ]
    ):
        state_dict = {
            k.replace(
                "frontend.upstream.model",
                "frontend.upstream.upstream.model",
            ): v
            for k, v in state_dict.items()
        }
        model.load_state_dict(state_dict, strict=not use_lora)

In [4]:
import yaml
import os

config_model_path = '/home/royshivam/anaconda3/envs/espnet/lib/python3.8/site-packages/espnet_model_zoo/models--pyf98--librispeech_100_e_branchformer/snapshots/e331ba7b520392b7e290b9c73bf1453967a440d2/exp/asr_train_asr_e_branchformer_size256_mlp1024_linear1024_e12_mactrue_edrop0.0_ddrop0.0_raw_en_bpe5000_sp/config.yaml'

with open(config_model_path, 'r') as file:
    config_data = yaml.safe_load(file)
tokens_list = config_data.get('token_list', [])
tokens_path = "tokens.txt"
with open(tokens_path, 'w') as file:
    for item in tokens_list:
        file.write(f"{item}\n")

In [5]:
from espnet2.text.build_tokenizer import build_tokenizer
from espnet2.text.token_id_converter import TokenIDConverter


def convert_text_to_tokens(text, bpe_model_path, token_list):
    tokenizer = build_tokenizer(
        token_type='bpe',
        bpemodel=bpe_model_path
    )
    tok_to_id = TokenIDConverter(
        token_list=token_list
    )
    tokens = tokenizer.text2tokens(text)
    list_of_toks = [curr[0] for curr in tokens]
    text_rep = tok_to_id.tokens2ids(list_of_toks)
    return torch.tensor(text_rep)

bpe_model_path = '/home/royshivam/anaconda3/envs/espnet/lib/python3.8/site-packages/espnet_model_zoo/models--pyf98--librispeech_100_e_branchformer/snapshots/e331ba7b520392b7e290b9c73bf1453967a440d2/data/en_token_list/bpe_unigram5000/bpe.model'
text = convert_text_to_tokens(test_dataset[0][2], bpe_model_path, tokens_path)

In [6]:
speech = test_dataset[0][0].numpy().squeeze()
speech = torch.tensor(speech)

speech = speech.unsqueeze(0)
lengths = speech.new_full([1], dtype=torch.long, fill_value=speech.size(1))

text = text
text_length = len(text)
text = text.unsqueeze(0)
text_length = torch.tensor([text_length])

In [7]:
out = model(
    speech = speech,
    speech_lengths = lengths,
    text = text,
    text_lengths = text_length
)

In [8]:
print(model.decoder.train_decoder.shape)
print(model.decoder.train_softmax.shape)
print(model.encoder.train_attention.shape)

torch.Size([0, 47, 256])
torch.Size([0, 47, 5000])
torch.Size([1, 260, 256])


In [ ]:
tokens

In [ ]:
test_dataset[0]

In [ ]:
from torch

class CEM_Model(